In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy

# Import Machine Learning Dependencies
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
import io
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
from io import StringIO

In [3]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [5]:
engine = create_engine("sqlite:///final_project")

In [6]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [7]:
results = client.get("xwdj-i9he", limit=1160000)

In [8]:
results_df = pd.DataFrame.from_records(results)

In [9]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_lat,sr_location_lat_long,sr_status_date,sr_location_street_name,sr_created_date,sr_location_city,sr_location_y,sr_location_map_tile,sr_location_zip_code,sr_location_street_number
1052868,"BEE CAVES RD &S MOPAC EXPY SVRD SB, AUSTIN, TX",8,Phone,2014-10-02T12:26:57.000,3101694.78,Closed,Traffic Signal - Maintenance,TRAVIS,584W,-97.78283591,...,30.26623624,"{'latitude': '30.26623624', 'needs_recoding': ...",2014-10-02T12:26:57.000,BEE CAVES RD &S MOPAC EXPY SVRD SB,2014-10-02T08:32:55.000,AUSTIN,10069796.22,MG22,78746,NaN
710996,"1100 REINLI ST 7, AUSTIN, TX 78723",4,Phone,2017-05-11T08:50:45.000,3126350.24738037,Closed,Austin Code - Request Code Officer,TRAVIS,556J,-97.70333257,...,30.31702006,"{'latitude': '30.31702006', 'needs_recoding': ...",2017-05-11T08:50:45.000,REINLI,2017-05-10T12:47:56.000,AUSTIN,10088845.0008989,ML26,78723,1100
581166,"S 1ST ST & BARTON SPRINGS RD, AUSTIN, TX 78704",9,Phone,2018-06-01T10:09:46.000,3112101.74736203,Closed,Traffic Signal - Maintenance,TRAVIS,615A,-97.75007216,...,30.25838165,"{'latitude': '30.25838165', 'needs_recoding': ...",2018-06-01T10:09:46.000,S 1ST ST & BARTON SPRINGS RD,2018-05-04T07:15:01.000,AUSTIN,10067179.0001311,MJ21,78704,NaN
51368,"6301 NICKLAUS PL, AUSTIN, TX 78746",8,Phone,2021-06-11T13:33:34.000,3083062.50110687,Closed,Street Light Issue- Address,TRAVIS,582M,-97.84148916,...,30.28102012,"{'latitude': '30.28102012', 'human_address': '...",2021-06-11T13:33:34.000,NICKLAUS,2021-06-10T16:41:26.000,AUSTIN,10074752.00245380,MD23,78746,6301
1146386,"5900 HARWILL CIR, AUSTIN, TX",NaN,Phone,2014-01-22T12:38:54.000,3132230.02257715,Closed,Loose Dog,TRAVIS,556Q,-97.68487456,...,30.31039523,"{'latitude': '30.31039523', 'needs_recoding': ...",2014-01-22T12:38:54.000,HARWILL,2014-01-20T10:32:02.000,AUSTIN,10086574.464919,MM26,78723,5900
149492,"BROKEN ARROW LN & MEADOW LEA DR, AUSTIN, TX",2,Phone,2020-11-30T07:40:31.000,3101765.00086965,Closed,Loose Dog,TRAVIS,674A,-97.78467612,...,30.18810038,"{'latitude': '30.18810038', 'human_address': '...",2020-11-30T07:40:31.000,BROKEN ARROW LN & MEADOW LEA DR,2020-11-29T17:25:21.000,AUSTIN,10041373.00127110,MG15,78745,NaN
268374,"945 E 51ST ST, AUSTIN, TX 78751",9,Spot311 Interface,2020-08-30T01:04:40.000,3123614.95169256,Closed,Pothole Repair,TRAVIS,555R,-97.71218018,...,30.31052968,"{'latitude': '30.31052968', 'human_address': '...",2020-08-30T01:04:40.000,51ST,2020-03-02T17:38:33.000,AUSTIN,10086419.28679730,MK26,78751,945
1122969,"7123 N IH 35 SVRD NB, AUSTIN, TX",NaN,Phone,2014-04-24T06:43:12.000,3126110.47758064,Closed,Street Light Issue- Address,TRAVIS,526W,-97.70367017,...,30.33252358,"{'latitude': '30.33252358', 'needs_recoding': ...",2014-04-24T06:43:12.000,35,2014-04-07T09:54:31.000,AUSTIN,10094479.368736,ML28,78752,7123
484859,"E WILLIAM CANNON DR & BLUFF SPRINGS RD, AUSTIN...",2,Phone,2018-12-01T00:04:09.000,3107055.56,Duplicate (closed),Traffic Signal - Maintenance,TRAVIS,674C,-97.7678978,...,30.18919452,"{'latitude': '30.18919452', 'needs_recoding': ...",2018-12-01T00:04:09.000,E WILLIAM CANNON DR & BLUFF SPRINGS RD,2018-11-30T08:38:00.000,AUSTIN,10041892.67,MH15,78744,NaN
441956,"AIRPORT BLVD & E MARTIN LUTHER KING JR BLVD, A...",1,Phone,2019-03-18T13:59:15.000,3126451.49730109,Duplicate (closed),Traffic Signal - Maintenance,TRAVIS,586J,-97.70395032,...,30.28257067,"{'latitude': '30.28257067', 'needs_recoding': ...",2019-03-18T13:59:15.000,AIRPORT BLVD & E MARTIN LUTHER KING JR BLVD,2019-03-18T13:53:58.000,Austin,10076315.0020443,ML23,78721,NaN


In [10]:
len(results_df)

1151553

In [11]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [12]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [13]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [14]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [15]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [16]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [17]:
median = results_df['Total Seconds'].median(axis=0)

In [18]:
#median

In [19]:
conditions = [
    (results_df['Total Seconds'] <= median),
    (results_df['Total Seconds'] > median)
    ]
values = ['Quick Response', 'Slow Response']
results_df['Response Time'] = np.select(conditions, values)

In [20]:
#print(results_df['Total Seconds'].max())

In [21]:
#results_df.loc[results_df['Total Seconds'] == 109367883]

In [22]:
#np.nanpercentile(results_df['Total Seconds'], 99.35) 

In [23]:
response = results_df['Response Time'].value_counts()
response

Slow Response     163163
Quick Response    163163
Name: Response Time, dtype: int64

In [24]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [25]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [26]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [27]:
#top_dept = results_df['Department'].value_counts()
#top_dept

In [28]:
#print(results_df.columns)

In [29]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [30]:
training_df = results_df.drop(columns=['sr_location', 'Open Time', 'sr_location_lat_long', 'sr_location_council_district', 'sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_long', 'sr_updated_date', 'sr_location_lat', 'sr_status_date', 'sr_location_street_name', 'sr_created_date', 'sr_location_street_number', 'sr_location_city', 'sr_location_y', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Open Time delta'])

In [31]:
training_df = training_df.drop(columns=['sr_type_desc','sr_number'])

In [32]:
#training_df = training_df.drop(columns=[])

In [33]:
training_df = training_df.rename(columns={'sr_method_received_desc':'Report Method', "sr_location_zip_code":"Zip Code"})

In [34]:
training_df.head(10)

,Report Method,Zip Code,Request Year,Total Seconds,Response Time,Department
295408,Phone,78723,2019,152190.0,Slow Response,Austin Code
295409,Spot311 Interface,78744,2019,152934.0,Slow Response,Austin Code
295410,Web,78701,2019,113327.0,Slow Response,Public Works
295411,Spot311 Interface,78701,2019,121696.0,Slow Response,Austin Resource Recovery
295412,Phone,78701,2019,155392.0,Slow Response,Austin Code
295413,Phone,78745,2019,18479378.0,Slow Response,Animal Services
295414,Spot311 Interface,78745,2019,155819.0,Slow Response,Austin Code
295415,Phone,78746,2019,1097446.0,Slow Response,Austin Resource Recovery
295416,Spot311 Interface,78745,2019,117788.0,Slow Response,Animal Services
295417,Spot311 Interface,78660,2019,162582.0,Slow Response,Animal Services


In [35]:
#training_df.rename(columns={'index':'Index'}, inplace =True)
#command = 'Index serial primary key, Report Method text, Zip Code varchar(5), Request Year int, Total Seconds interval, Response Time text, Department text'


training_df.to_sql('database', con=engine, if_exists='replace')

engine.execute("SELECT * FROM database").fetchall()

[(295408, 'Phone', '78723', 2019, 152190.0, 'Slow Response', 'Austin Code'),
 (295409, 'Spot311 Interface', '78744', 2019, 152934.0, 'Slow Response', 'Austin Code'),
 (295410, 'Web', '78701', 2019, 113327.0, 'Slow Response', 'Public Works'),
 (295411, 'Spot311 Interface', '78701', 2019, 121696.00000000001, 'Slow Response', 'Austin Resource Recovery'),
 (295412, 'Phone', '78701', 2019, 155392.0, 'Slow Response', 'Austin Code'),
 (295413, 'Phone', '78745', 2019, 18479378.0, 'Slow Response', 'Animal Services'),
 (295414, 'Spot311 Interface', '78745', 2019, 155819.0, 'Slow Response', 'Austin Code'),
 (295415, 'Phone', '78746', 2019, 1097446.0, 'Slow Response', 'Austin Resource Recovery'),
 (295416, 'Spot311 Interface', '78745', 2019, 117788.00000000001, 'Slow Response', 'Animal Services'),
 (295417, 'Spot311 Interface', '78660', 2019, 162582.0, 'Slow Response', 'Animal Services'),
 (295418, 'Spot311 Interface', '78757', 2019, 162712.0, 'Slow Response', 'Animal Services'),
 (295419, 'Phone'

Machine Learning Model

In [36]:
# Create our features
X = training_df.drop(columns = ["Response Time"])

# Create dummy variables
X = pd.get_dummies(X)

# Create our target
y = training_df[["Response Time"]]

In [37]:
# Check the balance of our target values
y["Response Time"].value_counts()

Slow Response     163163
Quick Response    163163
Name: Response Time, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [39]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_for_class = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
rand_for_class.fit(X_train, y_train.values.ravel())

BalancedRandomForestClassifier(random_state=1)

In [40]:
# Calculated the balanced accuracy score
y_prediction = rand_for_class.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

0.9999877621949727

In [41]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[40725,     0],
       [    1, 40856]], dtype=int64)

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                      pre       rec       spe        f1       geo       iba       sup

Quick Response       1.00      1.00      1.00      1.00      1.00      1.00     40725
 Slow Response       1.00      1.00      1.00      1.00      1.00      1.00     40857

   avg / total       1.00      1.00      1.00      1.00      1.00      1.00     81582



In [43]:
# List the features sorted in descending order by feature importance
features = rand_for_class.feature_importances_
sort_features = np.argsort(features)[::-1]

for i in sort_features:
    print("{}: ({})".format(X.columns.values[i], features[i]))

Total Seconds: (0.8199200955415215)
Department_Austin Code: (0.10911936345980736)
Department_Austin Resource Recovery: (0.022420757230637797)
Department_Austin Transportation: (0.009375291534289414)
Department_Watershed Check: (0.004970995026808016)
Department_Animal Services: (0.004845010574832926)
Report Method_Phone: (0.0042809026538513295)
Department_Public Works: (0.004101226195109957)
Report Method_Spot311 Interface: (0.00363657726094304)
Department_Parks and Recreation: (0.0022850302780916634)
Report Method_Web: (0.00219988062050061)
Department_Public Health: (0.0020719595887028898)
Zip Code_78701: (0.0020654004595478306)
Request Year: (0.0020103580644305016)
Department_Austin Water: (0.0013375763456474098)
Zip Code_78702: (0.000375256733068445)
Zip Code_78741: (0.0003125961411242852)
Zip Code_78758: (0.0002970271629739315)
Zip Code_78703: (0.0002559678811358992)
Zip Code_78753: (0.00024440964171287675)
Zip Code_78705: (0.0002183487559804042)
Zip Code_78749: (0.00019482219832465